In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.translate.meteor_score import meteor_score, single_meteor_score
import nltk

In [ ]:
!apt-get -y install protobuf-compiler python-pil python-lxml

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-pil
E: Unable to locate package python-lxml


In [ ]:
!pip install openNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 24.6 MB/s eta 0:00:00


In [ ]:
config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn-en.txt.subword.train
        path_tgt: /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn.txt.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn-en.txt.subword.dev
        path_tgt: /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn.txt.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 64
tgt_seq_length: 64

# Tokenization options
src_subword_model: /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/source.model
tgt_subword_model: /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/target.model
# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.engshona

# Stop training if it does not imporve after n validations
early_stopping: 10

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 2023

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 6000

# Default: 10000 - Run validation after n steps
valid_steps: 4000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 5000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 1024  # Tokens per batch, change when CUDA out of memory
valid_batch_size: 1024
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.4
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 16
dec_layers: 16
heads: 16
hidden_size: 1024
word_vec_size: 1024
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!onmt_build_vocab -config /content/config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-08-19 16:26:26,873 INFO] Counter vocab from -1 samples.
[2023-08-19 16:26:26,873 INFO] n_sample=-1: Build vocab on full datasets.
[2023-08-19 16:26:34,114 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=2738)

[2023-08-19 16:26:34,278 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=2731)

[2023-08-19 16:26:34,495 INFO] Counters src: 44634
[2023-08-19 16:26:34,495 INFO] Counters tgt: 46189


In [ ]:
 # Train the NMT model
!onmt_train -config /content/config.yaml

[2023-08-19 16:26:38,856 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-08-19 16:26:38,856 INFO] Parsed 2 corpora from -data.
[2023-08-19 16:26:38,857 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-08-19 16:26:39,238 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '.', '▁', '0', '▁the', '1']
[2023-08-19 16:26:39,238 INFO] The decoder start token is: <s>
[2023-08-19 16:26:39,239 INFO] Building model...
[2023-08-19 16:26:44,743 INFO] Switching model to float32 for amp/apex_amp
[2023-08-19 16:26:44,744 INFO] Non quantized layer compute is fp16
[2023-08-19 16:26:52,241 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(44640, 1024, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Mo

In [ ]:
!onmt_translate -model /content/models/model.engshona_step_6000.pt -src /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn-en.txt.subword.test -output /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/ShonaV9.translated

[2023-08-19 18:30:37,409 INFO] Loading checkpoint from /content/models/model.engshona_step_6000.pt
[2023-08-19 18:31:00,243 INFO] Loading data into the model
Traceback (most recent call last):
  File "/usr/local/bin/onmt_translate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/translate.py", line 56, in main
    translate(opt)
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/translate.py", line 37, in translate
    _, _ = translator._translate(
  File "/usr/local/lib/python3.10/dist-packages/onmt/translate/translator.py", line 441, in _translate
    self.out_file.flush()
OSError: [Errno 107] Transport endpoint is not connected


In [ ]:
!mkdir -p nmt
%cd nmt
!git clone https://github.com/ymoslem/MT-Preparation.git

/content/nmt
Cloning into 'MT-Preparation'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 239 (delta 119), reused 186 (delta 94), pack-reused 0
Receiving objects: 100% (239/239), 61.56 KiB | 4.74 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [ ]:
# Install the requirements
!pip3 install -r MT-Preparation/requirements.txt

In [ ]:
%cd  ../

In [ ]:
!ls

In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/target.model /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/ShonaV9.translated

In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/target.model /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn.txt.subword.test

In [ ]:

# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
!pip3 install sacrebleu

--2024-04-29 16:41:28--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2024-04-29 16:41:28 (44.3 MB/s) - ‘compute-bleu.py’ saved [957/957]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
!python3 compute-bleu.py /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn.txt.subword.test.desubword /content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/ShonaV9.translated.desubword

Reference 1st sentence: Akatswiri Amagetsi
MTed 1st sentence: Sarudza Mutauro
BLEU:  22.04878620393455


In [ ]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('punkt')

# Function to read the content of a text file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Tokenize the text
def tokenize(text):
    return nltk.word_tokenize(text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Function to read the content of a text file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Paths to your text files
file1_path = '/content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/ShonaV9.translated.desubword'
file2_path = '/content/drive/MyDrive/LanguageTranslationData_New/EnglishShona_DataPrep_SentencePieceModel/CCAligned.en-sn.txt.subword.test.desubword'

# Read the contents of the files
reference = read_file(file1_path)
hypothesis = read_file(file2_path)

# Tokenize the texts
reference_tokens = tokenize(reference)
hypothesis_tokens = tokenize(hypothesis)

# Calculate the METEOR score
score = meteor_score([reference_tokens], hypothesis_tokens)

print(f'The METEOR score between the two files is: {score}')

The METEOR score between the two files is: 0.2680294255395139


In [ ]:
#Observed inconsistency between the number of lines in the two files being compared.
#To check if the inconsistency is responsible for the declining METEOR score.